In [2]:
! apt-get install default-jre
!java -version

Reading package lists... Done
Building dependency tree       
Reading state information... Done
default-jre is already the newest version (2:1.11-68ubuntu1~18.04.1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 34 not upgraded.
openjdk version "11.0.10" 2021-01-19
OpenJDK Runtime Environment (build 11.0.10+9-Ubuntu-0ubuntu1.18.04)
OpenJDK 64-Bit Server VM (build 11.0.10+9-Ubuntu-0ubuntu1.18.04, mixed mode, sharing)


In [3]:
! pip install h2o

In [4]:
import h2o
from h2o.automl import H2OAutoML

In [5]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.10" 2021-01-19; OpenJDK Runtime Environment (build 11.0.10+9-Ubuntu-0ubuntu1.18.04); OpenJDK 64-Bit Server VM (build 11.0.10+9-Ubuntu-0ubuntu1.18.04, mixed mode, sharing)
  Starting server from /usr/local/lib/python3.7/dist-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmp6erm93id
  JVM stdout: /tmp/tmp6erm93id/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmp6erm93id/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,03 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.32.1.1
H2O_cluster_version_age:,30 days
H2O_cluster_name:,H2O_from_python_unknownUser_0r0d1b
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.180 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"accepting new members, healthy"


In [6]:
import os
docker_data_path = "C:/Users/VK SOLUTION/Desktop/powerplant_output.csv"
if os.path.isfile(docker_data_path):
  data_path = docker_data_path
else:
  data_path = "https://github.com/h2oai/h2o-tutorials/raw/master/h2o-world-2017/automl/data/powerplant_output.csv"


# Load data into H2O
df = h2o.import_file(data_path)

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [7]:
df.describe()

Rows:9568
Cols:5




,TemperatureCelcius,ExhaustVacuumHg,AmbientPressureMillibar,RelativeHumidity,HourlyEnergyOutputMW
type,real,real,real,real,real
mins,1.81,25.36,992.89,25.56,420.26
mean,19.651231187290957,54.3058037207358,1013.2590781772578,73.30897784280936,454.36500940635455
maxs,37.11,81.56,1033.3,100.16,495.76
sigma,7.452473229611082,12.707892998326807,5.93878370581162,14.600268756728957,17.066994999803423
zeros,0,0,0,0,0
missing,0,0,0,0,0
0,14.96,41.76,1024.07,73.17,463.26
1,25.18,62.96,1020.04,59.08,444.37
2,5.11,39.4,1012.16,92.14,488.56


In [8]:
y = "HourlyEnergyOutputMW"

In [9]:
splits = df.split_frame(ratios = [0.8], seed = 1)
train = splits[0]
test = splits[1]

In [10]:
aml = H2OAutoML(max_runtime_secs = 60, seed = 1, project_name = "Power_Planet_Data")
aml.train(y = y, training_frame = train, leaderboard_frame = test)

AutoML progress: |████████████████████████████████████████████████████████| 100%


In [12]:
# aml2 = H2OAutoML(max_runtime_secs = 60, seed = 1, project_name = "powerplant_full_data")
# aml2.train(y = y, training_frame = df)

AutoML progress: |████ (cancelled)


H2OJobCancelled: ignored

In [11]:
aml.leaderboard.head()

model_id,mean_residual_deviance,rmse,mse,mae,rmsle
XGBoost_grid__1_AutoML_20210425_093944_model_1,11.7239,3.42402,11.7239,2.34031,0.00751901
XGBoost_2_AutoML_20210425_093944,14.338,3.78656,14.338,2.77924,0.00832719
XGBoost_3_AutoML_20210425_093944,15.6642,3.95781,15.6642,2.97243,0.00870859
GBM_grid__1_AutoML_20210425_093944_model_1,16.1382,4.01724,16.1382,2.99246,0.00883251
XGBoost_grid__1_AutoML_20210425_093944_model_2,18.3797,4.28716,18.3797,3.24708,0.00940505
DeepLearning_1_AutoML_20210425_093944,21.4836,4.63504,21.4836,3.45221,0.010203
XRT_1_AutoML_20210425_093944,21.4955,4.63633,21.4955,3.28516,0.0102065
DeepLearning_grid__1_AutoML_20210425_093944_model_1,21.6627,4.65432,21.6627,3.5941,0.0102237
GLM_1_AutoML_20210425_093944,22.4247,4.73548,22.4247,3.74686,0.0103862
DRF_1_AutoML_20210425_093944,25.0549,5.00549,25.0549,3.46702,0.010972


In [12]:
aml.leader

Model Details
H2OXGBoostEstimator :  XGBoost
Model Key:  XGBoost_grid__1_AutoML_20210425_093944_model_1


Model Summary: 


,,number_of_trees
0,,51.0




ModelMetricsRegression: xgboost
** Reported on train data. **

MSE: 0.696920266176113
RMSE: 0.834817504713523
MAE: 0.640105247124884
RMSLE: 0.0018401629243108703
Mean Residual Deviance: 0.696920266176113

ModelMetricsRegression: xgboost
** Reported on cross-validation data. **

MSE: 11.046753516946008
RMSE: 3.323665674664949
MAE: 2.351983028012855
RMSLE: 0.0072905572947118335
Mean Residual Deviance: 11.046753516946008

Cross-Validation Metrics Summary: 


,,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
0,mae,2.3519886,0.06347698,2.3815114,2.279263,2.346959,2.3098922,2.4423175
1,mean_residual_deviance,11.04688,0.835572,11.529542,9.598401,11.053468,11.498491,11.554495
2,mse,11.04688,0.835572,11.529542,9.598401,11.053468,11.498491,11.554495
3,r2,0.96198237,0.0031758691,0.9603808,0.9674963,0.96174335,0.96068114,0.9596104
4,residual_deviance,11.04688,0.835572,11.529542,9.598401,11.053468,11.498491,11.554495
5,rmse,3.321691,0.12868698,3.3955178,3.0981286,3.3246756,3.3909426,3.3991902
6,rmsle,0.007286154,2.84574E-4,0.007444629,0.006799856,0.00726554,0.007435883,0.007484864



Scoring History: 


,,timestamp,duration,number_of_trees,training_rmse,training_mae,training_deviance
0,,2021-04-25 09:40:21,8.533 sec,0.0,454.182071,453.861607,206281.353311
1,,2021-04-25 09:40:21,8.604 sec,5.0,76.399160,76.294025,5836.831684
2,,2021-04-25 09:40:21,8.687 sec,10.0,13.000175,12.830564,169.004544
3,,2021-04-25 09:40:21,8.776 sec,15.0,2.753870,2.339737,7.583801
4,,2021-04-25 09:40:22,9.033 sec,20.0,1.535797,1.175908,2.358671
5,,2021-04-25 09:40:22,9.237 sec,25.0,1.354720,1.029858,1.835266
6,,2021-04-25 09:40:22,9.512 sec,30.0,1.219555,0.927821,1.487315
7,,2021-04-25 09:40:22,9.622 sec,35.0,1.128138,0.861064,1.272696
8,,2021-04-25 09:40:22,9.754 sec,40.0,1.024735,0.784401,1.050082
9,,2021-04-25 09:40:23,9.890 sec,45.0,0.938543,0.719709,0.880863



Variable Importances: 


,variable,relative_importance,scaled_importance,percentage
0,TemperatureCelcius,2.728356e+06,1.000000,0.785370
1,ExhaustVacuumHg,3.219147e+05,0.117989,0.092665
2,AmbientPressureMillibar,3.095895e+05,0.113471,0.089117
3,RelativeHumidity,1.141136e+05,0.041825,0.032848


In [13]:
aml.predict(test)

xgboost prediction progress: |████████████████████████████████████████████| 100%


predict
485.695
473.756
467.413
450.41
448.128
468.133
442.936
466.101
442.655
431.161


In [14]:
len(test)

1891

In [18]:
test.loc(0)

AttributeError: ignored

In [20]:
test_pd_df = test.as_data_frame()

In [21]:
test_pd_df.iloc[0]

TemperatureCelcius            5.11
ExhaustVacuumHg              39.40
AmbientPressureMillibar    1012.16
RelativeHumidity             92.14
HourlyEnergyOutputMW        488.56
Name: 0, dtype: float64

In [22]:
aml.leader.predict(test_pd_df.iloc[0])

ValueError: ignored

In [23]:
hf = h2o.H2OFrame(test_pd_df.iloc[0])

H2OTypeError: ignored

In [26]:
 test[range(0,20),:]

TemperatureCelcius,ExhaustVacuumHg,AmbientPressureMillibar,RelativeHumidity,HourlyEnergyOutputMW
5.11,39.4,1012.16,92.14,488.56
11.74,43.56,1015.14,70.72,477.5
17.76,42.42,1009.09,66.26,468.27
16.38,47.45,1010.08,88.86,450.69
23.18,51.3,1012.04,78.64,448.86
12.82,38.62,1018.71,83.8,467.41
23.04,59.43,1010.23,68.99,442.99
13.76,45.87,1008.89,87.27,463.5
25.5,58.79,1016.02,64.4,440.01
26.46,74.67,1016.65,84.44,431.65


In [27]:
aml.predict(test[range(0,20),:])

xgboost prediction progress: |████████████████████████████████████████████| 100%


predict
485.695
473.756
467.413
450.41
448.128
468.133
442.936
466.101
442.655
431.161
